In [63]:
# Take start# and end$ into consideration
import pandas as pd
import re
import math

def read_data():
    df = pd.read_excel('./prep_data.xlsx',header = 0)
    df = df[df.columns[1:]]
    words = df['Word'].to_list()
    for i in range(len(words)):
        words[i] = '#' + words[i] + '$'

    data = []
    for i in range(len(words)):
        for letter in list(words[i]):
            data.append(letter)

    return words,data

def create_bigram(data):
    bigram_list = []
    bigram_cnt = {}
    unigram_cnt = {}
    for i in range(len(data)):
        if i < len(data) - 1 and (i % 7 != 6):
            bigram_list.append( (data[i],data[i + 1]))

            if (data[i], data[i + 1]) in bigram_cnt:
                bigram_cnt[(data[i], data[i + 1])] += 1
            else:
                bigram_cnt[(data[i], data[i + 1])] = 1 
        
        if(data[i] in unigram_cnt):
            unigram_cnt[data[i]] += 1
        else:
            unigram_cnt[data[i]] = 1

    return bigram_list,bigram_cnt, unigram_cnt

def get_bigram_prob(bigram_list, bigram_cnt, unigram_cnt):
    bigram_prob = {}
    for bigram in bigram_list:
        letter1 = bigram[0]
        letter2 = bigram[1]
        bigram_prob[bigram] = (bigram_cnt[bigram]) / (unigram_cnt[letter1])

    return bigram_prob

def get_word_prob(word,bigram_prob):
    letters = list(word)
    word_bigram_list = []
    output_prob = 1

    for i in range(len(letters) - 1):
        word_bigram_list.append((letters[i], letters[i + 1]))

    for i in range(len(word_bigram_list)):
        if word_bigram_list[i] in bigram_prob:
            output_prob *= bigram_prob[word_bigram_list[i]]
        else:
            output_prob *= 0
        
    return output_prob

def output_data(words,bigram_prob):
    df = pd.read_excel('./prep_data.xlsx',header = 0)
    df = df[df.columns[1:]]
    prob_lists = []
    for i in range(len(words)):
        prob_lists.append(get_word_prob(words[i],bigram_prob))
    df['probability'] = pd.DataFrame(prob_lists)
    df.to_excel('./myprep_data.xlsx')

    return

## Main Function
words, data = read_data()
bigram_list, bigram_cnt, unigram_cnt = create_bigram(data)
bigram_prob = get_bigram_prob(bigram_list, bigram_cnt, unigram_cnt)
# output_data(words,bigram_prob)


In [45]:
import pandas as pd
import re

def read_data():
    df = pd.read_excel('./prep_data.xlsx',header = 0)
    df = df[df.columns[1:]]
    words = df['Word'].to_list()
    data = []

    for i in range(len(words)):
        for letter in re.findall('[a-z]',words[i]):
            data.append(letter)

    return words,data

def create_bigram(data):
    bigram_list = []
    bigram_cnt = {}
    unigram_cnt = {}
    for i in range(len(data)):
        if i < len(data) - 1 and (i % 5 != 4):
            bigram_list.append( (data[i],data[i + 1]))

            if (data[i], data[i + 1]) in bigram_cnt:
                bigram_cnt[(data[i], data[i + 1])] += 1
            else:
                bigram_cnt[(data[i], data[i + 1])] = 1 
        
        if(data[i] in unigram_cnt):
            unigram_cnt[data[i]] += 1
        else:
            unigram_cnt[data[i]] = 1

    return bigram_list,bigram_cnt, unigram_cnt

def get_bigram_prob(bigram_list, bigram_cnt, unigram_cnt):
    prob_list = {}
    for bigram in bigram_list:
        letter1 = bigram[0]
        letter2 = bigram[1]
        prob_list[bigram] = (bigram_cnt.get(bigram)) / (unigram_cnt.get(letter1))

    return prob_list


def get_word_bigram(word):
    letters = list(word)
    word_bigram_list = []
    for i in range(len(letters) - 1):
        if i < len(letters) - 1:
            word_bigram_list.append((letters[i], letters[i + 1]))
    return word_bigram_list

def get_word_prob(word,bigram_prob):
    word_bigram_list = get_word_bigram(word)
    output_prob = 1

    for i in range(len(word_bigram_list)):
        if word_bigram_list[i] in bigram_prob:
            output_prob *= bigram_prob[word_bigram_list[i]]
        else:
            output_prob *= 0
    return output_prob

def get_phoneme_cnt(words):
    vowels = ['a','e','i','o','u']
    vowel_cnt = []
    consont_cnt = []
    for i in range(len(words)):
        vowel_cnt.append(0)
        consont_cnt.append(0)
    
    for i in range(len(words)):
        for letter in list(words[i]):
            if letter in vowels:
                vowel_cnt[i] += 1
            else:
                consont_cnt[i] += 1

    return vowel_cnt, consont_cnt

def get_unigram_freq(words, unigram_cnt):

    letters_sum = 0
    for val in unigram_cnt.values():
        letters_sum += val

    letter_freq = []
    for i in range(len(words)):
        letter_freq.append(0)
        for letter in list(words[i]):
            letter_freq[i] += unigram_cnt[letter]

    return letter_freq

def get_bigram_freq(words, bigram_cnt):
    bigram_freq = [[],[],[],[]]

    for word in words:
        word_bigram_list = get_word_bigram(word)
        for i in range(len(word_bigram_list)):
            freq = bigram_cnt[word_bigram_list[i]]
            bigram_freq[i].append(freq)

    return bigram_freq
## Main Function
words, data = read_data()
bigram_list, bigram_cnt, unigram_cnt = create_bigram(data)


bigram_prob = get_bigram_prob(bigram_list, bigram_cnt, unigram_cnt)
get_unigram_freq(words, unigram_cnt)
get_bigram_freq(words, bigram_cnt)

[7, 5, 9, 8, 5, 3, 4, 8, 19, 13, 2, 9, 8, 8, 8, 7, 10, 19, 8, 9, 4, 5, 10, 2, 11, 21, 4, 13, 2, 10, 28, 13, 16, 11, 28, 14, 8, 7, 1, 9, 9, 10, 9, 28, 3, 13, 9, 8, 10, 9, 9, 5, 6, 3, 9, 12, 16, 4, 9, 16, 11, 17, 3, 6, 4, 11, 28, 7, 14, 8, 3, 9, 11, 9, 21, 8, 2, 5, 5, 10, 13, 7, 7, 9, 28, 2, 20, 7, 3, 13, 9, 8, 9, 20, 5, 8, 13, 1, 8, 9, 28, 5, 8, 9, 13, 6, 3, 21, 3, 6, 4, 5, 8, 8, 26, 28, 9, 21, 4, 12, 1, 8, 10, 1, 20, 5, 13, 10, 4, 3, 28, 11, 3, 9, 4, 17, 9, 14, 4, 14, 10, 16, 5, 1, 17, 16, 16, 7, 3, 9, 16, 6, 11, 13, 16, 12, 10, 9, 3, 2, 28, 7, 13, 7, 5, 14, 4, 6, 13, 10, 4, 17, 10, 9, 7, 28, 5, 2, 9, 8, 9, 16, 8, 2, 5, 28, 9, 2, 4, 8, 8, 14, 8, 21, 9, 28, 9, 17, 10, 7, 9, 2, 7, 6, 7, 16, 13, 4, 6, 19, 9, 6, 10, 14, 13, 4, 2, 28, 13, 8, 10, 4, 13, 14, 9, 6, 4, 3, 4, 28, 4, 1, 9, 4, 3, 1, 7, 28, 3, 9, 8, 3, 13, 6, 2, 4, 14, 13, 1, 28, 14, 8, 8, 9, 11, 9, 8, 14, 9, 3, 4, 1, 5, 3, 13, 12, 10, 3, 1, 16, 4, 16, 4, 4, 8, 8, 20, 16, 17, 8, 26, 21, 2, 5, 13, 13, 2, 4, 4, 7, 8, 5, 13, 1, 7, 3, 

In [46]:
print(bigram_cnt[('m','p')])
print(bigram_cnt[('g','o')])

7
5
